In [1]:
import pandas as pd
import numpy as np
import ast
import pystan
import arviz as az

# Load the data
df = pd.read_csv('Elicitation Formats/rank-rank/rank-rank_1_4.csv')

# Convert string representations of lists into actual Python lists
df['options'] = df['options'].apply(ast.literal_eval)
df['votes'] = df['votes'].apply(ast.literal_eval)
df['predictions'] = df['predictions'].apply(ast.literal_eval)

# Convert lists in 'options' to tuples for hashability
df['options'] = df['options'].apply(tuple)

# Function to preprocess and convert rankings to indices
def convert_rankings_to_indices(rankings, options):
    option_to_index = {option: i + 1 for i, option in enumerate(sorted(options))}
    return [option_to_index[item] for item in rankings]

# Group by options
grouped = df.groupby('options')

# Dictionary to store models and fits for each option set
models = {}
fits = {}
rankings = {}

# Stan model code
stan_model_code = """
functions {
    real plackett_luce_lpmf(int[] ranking, vector strengths, int[] ground_truth) {
        real log_p = 0;
        for (k in 1:size(ranking)) {
            int item = ground_truth[ranking[k]];
            real sum_strengths = 0;
            for (m in k:size(ranking)) {
                int idx = ground_truth[ranking[m]];
                sum_strengths += strengths[idx];
            }
            log_p += log(strengths[item] / sum_strengths);
        }
        return log_p;
    }
}

data {
    int<lower=1> J;  // Number of questions
    int<lower=1> K;  // Number of options per question
    int<lower=1> N;  // Total number of unique options
    int<lower=1, upper=N> vote_indices[J, K]; // Indices of votes
    int<lower=1, upper=N> prediction_indices[J, K]; // Indices of predictions
    int<lower=1, upper=N> ground_truth_indices[J, K]; // Indices of ground truth
}

parameters {
    simplex[N] strengths_expert_vote;     // Normalized strength parameters for expert votes
    simplex[N] strengths_nonexpert_vote;  // Normalized strength parameters for non-expert votes
    simplex[N] strengths_expert_prediction;     // Normalized strength parameters for expert predictions
    simplex[N] strengths_nonexpert_prediction;  // Normalized strength parameters for non-expert predictions
    simplex[2] pi;                   // Mixing proportions for expert and non-expert
}

model {
    // Priors for votes and predictions
    strengths_expert_vote ~ dirichlet(rep_vector(3, N));
    strengths_nonexpert_vote ~ dirichlet(rep_vector(1, N));
    strengths_expert_prediction ~ dirichlet(rep_vector(1, N));
    strengths_nonexpert_prediction ~ dirichlet(rep_vector(1, N));

    pi ~ dirichlet([1, 3]');

    // Prior constraints for votes
    for (i in 1:(N-1)) {
        target += normal_lpdf(strengths_expert_vote[i+1] | strengths_expert_vote[i] - 0.05, 0.025);
        target += normal_lpdf(sum(strengths_expert_vote[1:i]) | sum(strengths_nonexpert_vote[1:i]) + (N-i-1)*0.05, 0.025);

        // Prior constraints for predictions (same logic as votes)
        target += normal_lpdf(strengths_expert_prediction[i+1] | strengths_expert_prediction[i] - 0.05, 0.025);
        target += normal_lpdf(sum(strengths_expert_prediction[1:i]) | sum(strengths_nonexpert_prediction[1:i]) + (N-i-1)*0.05, 0.025);
    }

    // Likelihood for votes
    for (j in 1:J) {
        vector[2] log_probs_votes;
        vector[2] log_probs_predictions;

        log_probs_votes[1] = log(pi[1]) + plackett_luce_lpmf(vote_indices[j] | strengths_expert_vote, ground_truth_indices[j]);
        log_probs_votes[2] = log(pi[2]) + plackett_luce_lpmf(vote_indices[j] | strengths_nonexpert_vote, ground_truth_indices[j]);

        log_probs_predictions[1] = log(pi[1]) + plackett_luce_lpmf(prediction_indices[j] | strengths_expert_prediction, ground_truth_indices[j]);
        log_probs_predictions[2] = log(pi[2]) + plackett_luce_lpmf(prediction_indices[j] | strengths_nonexpert_prediction, ground_truth_indices[j]);

        target += log_sum_exp(log_probs_votes);
        target += log_sum_exp(log_probs_predictions);
    }
}

generated quantities {
    vector[J] log_lik_votes;
    vector[J] log_lik_predictions;
    for (j in 1:J) {
        vector[2] log_probs_votes;
        vector[2] log_probs_predictions;

        log_probs_votes[1] = log(pi[1]) + plackett_luce_lpmf(vote_indices[j] | strengths_expert_vote, ground_truth_indices[j]);
        log_probs_votes[2] = log(pi[2]) + plackett_luce_lpmf(vote_indices[j] | strengths_nonexpert_vote, ground_truth_indices[j]);

        log_probs_predictions[1] = log(pi[1]) + plackett_luce_lpmf(prediction_indices[j] | strengths_expert_prediction, ground_truth_indices[j]);
        log_probs_predictions[2] = log(pi[2]) + plackett_luce_lpmf(prediction_indices[j] | strengths_nonexpert_prediction, ground_truth_indices[j]);

        log_lik_votes[j] = log_sum_exp(log_probs_votes);
        log_lik_predictions[j] = log_sum_exp(log_probs_predictions);
    }
}

"""

# Loop through each group
for options, group in grouped:
    # Apply the function to convert rankings to indices
    group['vote_indices'] = group['votes'].apply(lambda x: convert_rankings_to_indices(x, options))
    group['prediction_indices'] = group['predictions'].apply(lambda x: convert_rankings_to_indices(x, options))
    group['ground_truth_indices'] = group['options'].apply(lambda x: convert_rankings_to_indices(x, options))

    # Prepare data for the Stan model
    stan_data = {
        'J': len(group),
        'K': len(options),
        'N': len(options),
        'vote_indices': np.array(group['vote_indices'].tolist()),
        'prediction_indices': np.array(group['prediction_indices'].tolist()),
        'ground_truth_indices': np.array(group['ground_truth_indices'].tolist())
    }

    # Compile and fit the model for each group
    sm = pystan.StanModel(model_code=stan_model_code)
    fit = sm.sampling(data=stan_data, iter=2000, chains=4)
    fits[str(options)] = fit

    # Extract the median of the posterior distributions for strengths
    expert_strengths = np.median(fit['strengths_expert_vote'], axis=0)
    nonexpert_strengths = np.median(fit['strengths_nonexpert_vote'], axis=0)

    # Generate rankings based on strengths
    expert_ranking = sorted(zip(options, expert_strengths), key=lambda x: x[1], reverse=True)
    nonexpert_ranking = sorted(zip(options, nonexpert_strengths), key=lambda x: x[1], reverse=True)

    # Store rankings
    rankings[str(options)] = {
        'expert_ranking': [x[0] for x in expert_ranking],
        'nonexpert_ranking': [x[0] for x in nonexpert_ranking]
    }

# Print rankings for each group
for options, ranks in rankings.items():
    print(f"Options set: {options}")
    print(f"Expert Rankings: {ranks['expert_ranking']}")
    print(f"Non-Expert Rankings: {ranks['nonexpert_ranking']}")
    print()

# Optionally, convert the fit object to arviz InferenceData
idata = az.from_pystan(posterior=fit)
models[str(options)] = idata


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0c

Options set: (1, 7, 13, 19)
Expert Rankings: [1, 7, 13, 19]
Non-Expert Rankings: [7, 13, 1, 19]

Options set: (2, 8, 14, 20)
Expert Rankings: [2, 8, 14, 20]
Non-Expert Rankings: [8, 2, 14, 20]

Options set: (3, 9, 15, 21)
Expert Rankings: [3, 9, 15, 21]
Non-Expert Rankings: [9, 3, 15, 21]

Options set: (5, 11, 17, 23)
Expert Rankings: [5, 11, 17, 23]
Non-Expert Rankings: [11, 17, 5, 23]

Options set: (7, 13, 19, 25)
Expert Rankings: [7, 13, 19, 25]
Non-Expert Rankings: [13, 19, 7, 25]

Options set: (9, 15, 21, 27)
Expert Rankings: [9, 15, 21, 27]
Non-Expert Rankings: [21, 15, 9, 27]

Options set: (11, 17, 23, 29)
Expert Rankings: [11, 17, 23, 29]
Non-Expert Rankings: [17, 11, 23, 29]

Options set: (12, 18, 24, 30)
Expert Rankings: [12, 18, 24, 30]
Non-Expert Rankings: [18, 24, 12, 30]

Options set: (13, 19, 25, 31)
Expert Rankings: [13, 19, 25, 31]
Non-Expert Rankings: [19, 25, 13, 31]

Options set: (15, 21, 27, 33)
Expert Rankings: [15, 21, 27, 33]
Non-Expert Rankings: [21, 27, 15, 33

In [2]:
import pandas as pd
import numpy as np
import random
from scipy.stats import norm
from itertools import permutations
from typing import List, Tuple
from tqdm import tqdm
import ast  # to parse keys from fits

# -----------------------------
# Define Simulation Classes
# -----------------------------

class Vote:
    def __init__(self, question_number: int, options: List[int], ranking: List[int], predicted_probs: dict, is_expert: bool):
        self.question_number = question_number
        self.options = options
        self.ranking = ranking
        self.predicted_probs = predicted_probs
        self.is_expert = is_expert

class Voter:
    def __init__(self, is_expert: bool, central_strengths: List[float], variance_expert: float, variance_nonexpert: float):
        self.is_expert = is_expert
        self.strength_params = self.sample_strength_params(central_strengths, variance_expert, variance_nonexpert)

    def sample_strength_params(self, central_strengths: List[float], variance_expert: float, variance_nonexpert: float):
        if self.is_expert:
            strengths = [abs(norm.rvs(loc=s, scale=variance_expert)) for s in central_strengths]
        else:
            strengths = [abs(norm.rvs(loc=s, scale=variance_nonexpert)) for s in central_strengths]
        total_strength = sum(strengths)
        return [s / total_strength for s in strengths]

    def vote(self, question_number: int, options: List[int], ground_truth: List[int], all_worlds: List[Tuple[int]]):
        signal = self.plackett_luce_vote(options)
        conditional_probs = {
            s_j: compute_conditional_prob(s_j, tuple(signal), all_worlds, ground_truth, self.is_expert, self.strength_params)
            for s_j in all_worlds
        }
        predicted_probs = self.predict(signal, conditional_probs, all_worlds)
        prediction = max(predicted_probs, key=predicted_probs.get)
        return Vote(question_number, options, signal, prediction, self.is_expert)

    def plackett_luce_vote(self, options):
        remaining_options = options.copy()
        ranking = []
        while remaining_options:
            current_strengths = [self.get_strength(opt) for opt in remaining_options]
            probs = [strength / sum(current_strengths) for strength in current_strengths]
            chosen = np.random.choice(remaining_options, p=probs)
            ranking.append(chosen)
            remaining_options.remove(chosen)
        return ranking

    def get_strength(self, option):
        return self.strength_params[option % len(self.strength_params)]

    def predict(self, signal, conditional_probs, all_worlds):
        prediction_probs = {world: prob for world, prob in conditional_probs.items() if world != tuple(signal)}
        total_prob = sum(prediction_probs.values())
        return {world: prob / total_prob for world, prob in prediction_probs.items()}


# -----------------------------
# Supporting Functions
# -----------------------------

computed_posteriors = {}

def compute_posterior(signal, world, all_worlds, ground_truth, is_expert, strength_params):
    key = (tuple(signal), tuple(world), tuple(ground_truth), is_expert)
    if key in computed_posteriors:
        return computed_posteriors[key]
    prior = 1 / len(all_worlds)
    likelihood = plackett_luce_probability(signal, world, strength_params)
    total_signal_prob = sum(plackett_luce_probability(signal, w, strength_params) * prior for w in all_worlds)
    posterior = likelihood * prior / total_signal_prob
    computed_posteriors[key] = posterior
    return posterior

def compute_conditional_prob(s_j, s_k, all_worlds, ground_truth, is_expert, strength_params, num_samples=500):
    weights = [compute_posterior(s_k, w, all_worlds, ground_truth, is_expert, strength_params) for w in all_worlds]
    sampled_worlds = random.choices(all_worlds, weights=weights, k=num_samples)
    return sum(plackett_luce_probability(s_j, w, strength_params) for w in sampled_worlds) / num_samples

def plackett_luce_probability(signal, world, strength_params):
    prob = 1.0
    remaining_options = list(world)
    for item in signal:
        if item in remaining_options:
            current_strengths = [strength_params[opt % len(strength_params)] for opt in remaining_options]
            total_strength = sum(current_strengths)
            idx = remaining_options.index(item)
            prob *= current_strengths[idx] / total_strength
            remaining_options.pop(idx)
    return prob

def simulate_voting(num_voters: int, subset: List[int], ground_truth: List[int], central_strengths: List[float], variance_expert: float, variance_nonexpert: float, prob_expert: float) -> List[Vote]:
    votes = []
    question_number = 1
    all_worlds = list(permutations(subset))
    num_experts = np.random.binomial(num_voters, prob_expert)
    voters = [Voter(False, central_strengths, variance_expert, variance_nonexpert) for _ in range(num_voters)]
    expert_indices = np.random.choice(num_voters, size=num_experts, replace=False)
    for idx in expert_indices:
        voters[idx].is_expert = True
    for voter in tqdm(voters, desc="Simulating Votes"):
        vote = voter.vote(question_number, subset, ground_truth, all_worlds)
        votes.append(vote)
    return votes

# -----------------------------
# Main Generator from Fits
# -----------------------------

def generate_and_combine_synthetic_votes(fits_dict, num_voters, output_path):
    all_votes = []
    for key, fit in fits_dict.items():
        print(f"📦 Generating data for option set: {key}")
        try:
            # Extract median strengths and normalize
            central_strengths = np.median(fit['strengths_expert_vote'], axis=0).tolist()
            central_strengths = [s / sum(central_strengths) for s in central_strengths]

            # Empirical variance from posterior samples
            variance_expert = np.mean(np.var(fit['strengths_expert_vote'], axis=0))
            variance_nonexpert = np.mean(np.var(fit['strengths_nonexpert_vote'], axis=0))

            # Expert mixing proportion from pi
            prob_expert = np.median(fit['pi'][:, 0])

            subset = list(ast.literal_eval(key))
            ground_truth = sorted(subset)

            # Simulate
            votes = simulate_voting(
                num_voters=num_voters,
                subset=subset,
                ground_truth=ground_truth,
                central_strengths=central_strengths,
                variance_expert=variance_expert,
                variance_nonexpert=variance_nonexpert,
                prob_expert=prob_expert
            )

            # Collect all
            for vote in votes:
                all_votes.append([
                    vote.question_number,
                    vote.options,
                    vote.ranking,
                    vote.predicted_probs,
                    vote.is_expert,
                    "Synthetic",
                    "PL"
                ])
        except Exception as e:
            print(f"⚠️ Skipping {key} due to error: {e}")

    # Combine into one DataFrame
    df = pd.DataFrame(all_votes, columns=['question', 'options', 'votes', 'predictions', 'is_expert', 'domain', 'treatment'])
    df.to_csv(output_path, index=False)
    print(f"\n✅ All synthetic votes saved to {output_path}")


# -----------------------------
# Execute Simulation
# -----------------------------

if __name__ == "__main__":
    output_path = "combined_plackett_luce_synthetic_votes.csv"
    num_voters = 1000  # can adjust if needed

    generate_and_combine_synthetic_votes(
        fits_dict=fits,
        num_voters=num_voters,
        output_path=output_path
    )


📦 Generating data for option set: (1, 7, 13, 19)


Simulating Votes: 100%|██████████| 1000/1000 [01:38<00:00, 10.12it/s]


📦 Generating data for option set: (2, 8, 14, 20)


Simulating Votes: 100%|██████████| 1000/1000 [01:34<00:00, 10.53it/s]


📦 Generating data for option set: (3, 9, 15, 21)


...

📦 Generating data for option set: (31, 37, 43, 49)


Simulating Votes: 100%|██████████| 1000/1000 [01:34<00:00, 10.55it/s]


📦 Generating data for option set: (32, 38, 44, 50)


Simulating Votes: 100%|██████████| 1000/1000 [01:35<00:00, 10.48it/s]



✅ All synthetic votes saved to combined_plackett_luce_synthetic_votes.csv


In [ ]:
import pandas as pd
import numpy as np
import ast
import pystan
import arviz as az

# Load the data
df = pd.read_csv('combined_plackett_luce_synthetic_votes.csv')


# Convert string representations of lists into actual Python lists
df['options'] = df['options'].apply(ast.literal_eval)
df['votes'] = df['votes'].apply(ast.literal_eval)
df['predictions'] = df['predictions'].apply(ast.literal_eval)

# Convert lists in 'options' to tuples for hashability
df['options'] = df['options'].apply(tuple)

# Function to preprocess and convert rankings to indices
def convert_rankings_to_indices(rankings, options):
    option_to_index = {option: i + 1 for i, option in enumerate(sorted(options))}
    return [option_to_index[item] for item in rankings]

# Group by options
grouped = df.groupby('options')

# Dictionary to store models and fits for each option set
models = {}
synth_fits = {}
rankings = {}

# Stan model code
stan_model_code = """
functions {
    real plackett_luce_lpmf(int[] ranking, vector strengths, int[] ground_truth) {
        real log_p = 0;
        for (k in 1:size(ranking)) {
            int item = ground_truth[ranking[k]];
            real sum_strengths = 0;
            for (m in k:size(ranking)) {
                int idx = ground_truth[ranking[m]];
                sum_strengths += strengths[idx];
            }
            log_p += log(strengths[item] / sum_strengths);
        }
        return log_p;
    }
}

data {
    int<lower=1> J;  // Number of questions
    int<lower=1> K;  // Number of options per question
    int<lower=1> N;  // Total number of unique options
    int<lower=1, upper=N> vote_indices[J, K]; // Indices of votes
    int<lower=1, upper=N> prediction_indices[J, K]; // Indices of predictions
    int<lower=1, upper=N> ground_truth_indices[J, K]; // Indices of ground truth
}

parameters {
    simplex[N] strengths_expert_vote;     // Normalized strength parameters for expert votes
    simplex[N] strengths_nonexpert_vote;  // Normalized strength parameters for non-expert votes
    simplex[N] strengths_expert_prediction;     // Normalized strength parameters for expert predictions
    simplex[N] strengths_nonexpert_prediction;  // Normalized strength parameters for non-expert predictions
    simplex[2] pi;                   // Mixing proportions for expert and non-expert
}

model {
    // Priors for votes and predictions
    strengths_expert_vote ~ dirichlet(rep_vector(3, N));
    strengths_nonexpert_vote ~ dirichlet(rep_vector(1, N));
    strengths_expert_prediction ~ dirichlet(rep_vector(1, N));
    strengths_nonexpert_prediction ~ dirichlet(rep_vector(1, N));

    pi ~ dirichlet([1, 3]');

    // Prior constraints for votes
    for (i in 1:(N-1)) {
        target += normal_lpdf(strengths_expert_vote[i+1] | strengths_expert_vote[i] - 0.05, 0.025);
        target += normal_lpdf(sum(strengths_expert_vote[1:i]) | sum(strengths_nonexpert_vote[1:i]) + (N-i-1)*0.05, 0.025);

        // Prior constraints for predictions (same logic as votes)
        target += normal_lpdf(strengths_expert_prediction[i+1] | strengths_expert_prediction[i] - 0.05, 0.025);
        target += normal_lpdf(sum(strengths_expert_prediction[1:i]) | sum(strengths_nonexpert_prediction[1:i]) + (N-i-1)*0.05, 0.025);
    }

    // Likelihood for votes
    for (j in 1:J) {
        vector[2] log_probs_votes;
        vector[2] log_probs_predictions;

        log_probs_votes[1] = log(pi[1]) + plackett_luce_lpmf(vote_indices[j] | strengths_expert_vote, ground_truth_indices[j]);
        log_probs_votes[2] = log(pi[2]) + plackett_luce_lpmf(vote_indices[j] | strengths_nonexpert_vote, ground_truth_indices[j]);

        log_probs_predictions[1] = log(pi[1]) + plackett_luce_lpmf(prediction_indices[j] | strengths_expert_prediction, ground_truth_indices[j]);
        log_probs_predictions[2] = log(pi[2]) + plackett_luce_lpmf(prediction_indices[j] | strengths_nonexpert_prediction, ground_truth_indices[j]);

        target += log_sum_exp(log_probs_votes);
        target += log_sum_exp(log_probs_predictions);
    }
}

generated quantities {
    vector[J] log_lik_votes;
    vector[J] log_lik_predictions;
    for (j in 1:J) {
        vector[2] log_probs_votes;
        vector[2] log_probs_predictions;

        log_probs_votes[1] = log(pi[1]) + plackett_luce_lpmf(vote_indices[j] | strengths_expert_vote, ground_truth_indices[j]);
        log_probs_votes[2] = log(pi[2]) + plackett_luce_lpmf(vote_indices[j] | strengths_nonexpert_vote, ground_truth_indices[j]);

        log_probs_predictions[1] = log(pi[1]) + plackett_luce_lpmf(prediction_indices[j] | strengths_expert_prediction, ground_truth_indices[j]);
        log_probs_predictions[2] = log(pi[2]) + plackett_luce_lpmf(prediction_indices[j] | strengths_nonexpert_prediction, ground_truth_indices[j]);

        log_lik_votes[j] = log_sum_exp(log_probs_votes);
        log_lik_predictions[j] = log_sum_exp(log_probs_predictions);
    }
}

"""

# Loop through each group
for options, group in grouped:
    # Apply the function to convert rankings to indices
    group['vote_indices'] = group['votes'].apply(lambda x: convert_rankings_to_indices(x, options))
    group['prediction_indices'] = group['predictions'].apply(lambda x: convert_rankings_to_indices(x, options))
    group['ground_truth_indices'] = group['options'].apply(lambda x: convert_rankings_to_indices(x, options))

    # Prepare data for the Stan model
    stan_data = {
        'J': len(group),
        'K': len(options),
        'N': len(options),
        'vote_indices': np.array(group['vote_indices'].tolist()),
        'prediction_indices': np.array(group['prediction_indices'].tolist()),
        'ground_truth_indices': np.array(group['ground_truth_indices'].tolist())
    }

    # Compile and fit the model for each group
    sm = pystan.StanModel(model_code=stan_model_code)
    synth_fit = sm.sampling(data=stan_data, iter=2000, chains=4)
    synth_fits [str(options)] = synth_fit

    # Extract the median of the posterior distributions for strengths
    expert_strengths = np.median(synth_fit['strengths_expert_vote'], axis=0)
    nonexpert_strengths = np.median(synth_fit['strengths_nonexpert_vote'], axis=0)

    # Generate rankings based on strengths
    expert_ranking = sorted(zip(options, expert_strengths), key=lambda x: x[1], reverse=True)
    nonexpert_ranking = sorted(zip(options, nonexpert_strengths), key=lambda x: x[1], reverse=True)

    # Store rankings
    rankings[str(options)] = {
        'expert_ranking': [x[0] for x in expert_ranking],
        'nonexpert_ranking': [x[0] for x in nonexpert_ranking]
    }

# Print rankings for each group
for options, ranks in rankings.items():
    print(f"Options set: {options}")
    print(f"Expert Rankings: {ranks['expert_ranking']}")
    print(f"Non-Expert Rankings: {ranks['nonexpert_ranking']}")
    print()

# Optionally, convert the fit object to arviz InferenceData
idata = az.from_pystan(posterior=synth_fit)
models[str(options)] = idata


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2ac0d2d41277f784ec115a700 NOW.
To run all diagnostics call pystan.check_hmc_diagnostics(fit)
INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_0c0cbbf2

In [ ]:
def extract_mean_params(fit):
    summary = fit.summary()
    param_names = summary['summary_rownames']
    means = summary['summary'][:, 0]  # First column is the mean
    return dict(zip(param_names, means))


In [ ]:
import pandas as pd

comparison_records = []

# Loop over each options group that exists in both
for key in fits.keys():
    print(f"\n📊 Comparing parameter recovery for option set: {key}")
    try:
        original_params = extract_mean_params(fits[key])
        synthetic_params = extract_mean_params(synth_fits[key])

        for param in ['pi[1]', 'pi[2]'] + \
                     [f'strengths_expert_vote[{i+1}]' for i in range(len(original_params) // 6)]:
            rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
            comparison_records.append({
                'options': key,
                'parameter': param,
                'original': original_params[param],
                'synthetic': synthetic_params[param],
                'relative_error': rel_error
            })
            print(f"{param}: Relative Error = {rel_error:.4f}")
    except Exception as e:
        print(f"⚠️ Could not compare for {key}: {e}")


In [ ]:
df_compare = pd.DataFrame(comparison_records)
df_compare.to_csv('parameter_recovery_comparison.csv', index=False)


In [ ]:
import numpy as np
import pandas as pd

# Initialize dictionaries to store relative errors for each parameter group
relative_errors = {
    'expert_vote': [],
    'nonexpert_vote': [],
    'expert_prediction': [],
    'nonexpert_prediction': [],
    'expert_proportion': []
}

# Iterate over each group of options
for key in fits.keys():
    try:
        original_params = extract_mean_params(fits[key])
        synthetic_params = extract_mean_params(synth_fits[key])

        # Determine the number of options (N)
        N = sum(1 for p in original_params if p.startswith('strengths_expert_vote'))

        # Compute relative errors for each parameter group
        for i in range(N):
            # Expert vote strengths
            param = f'strengths_expert_vote[{i+1}]'
            rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
            relative_errors['expert_vote'].append(rel_error)

            # Non-expert vote strengths
            param = f'strengths_nonexpert_vote[{i+1}]'
            rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
            relative_errors['nonexpert_vote'].append(rel_error)

            # Expert prediction strengths
            param = f'strengths_expert_prediction[{i+1}]'
            rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
            relative_errors['expert_prediction'].append(rel_error)

            # Non-expert prediction strengths
            param = f'strengths_nonexpert_prediction[{i+1}]'
            rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
            relative_errors['nonexpert_prediction'].append(rel_error)

        # Expert proportion (pi[1])
        rel_error_pi = abs(original_params['pi[1]'] - synthetic_params['pi[1]']) / original_params['pi[1]']
        relative_errors['expert_proportion'].append(rel_error_pi)

    except Exception as e:
        print(f"⚠️ Could not compare for {key}: {e}")


In [ ]:
# Compute average relative errors
average_errors = {param: np.mean(errors) for param, errors in relative_errors.items()}

# Display the results
print("\n📊 Aggregate Parameter Recovery Errors:")
for param_group, avg_error in average_errors.items():
    print(f"🔹 {param_group.replace('_', ' ').title()}: {avg_error:.4f}")


In [ ]:
# Convert the results to a DataFrame
df_summary = pd.DataFrame([
    {'Parameter Group': param_group.replace('_', ' ').title(), 'Average Relative Error': avg_error}
    for param_group, avg_error in average_errors.items()
])

# Save to CSV
df_summary.to_csv('aggregate_parameter_recovery_summary.csv', index=False)


In [ ]:
import pandas as pd
import numpy as np
import ast
import pystan
import arviz as az

# Load the data
df = pd.read_csv('Elicitation Formats/rank-rank/rank-rank_1_4.csv')
# Convert string representations of lists into actual Python lists
df['options'] = df['options'].apply(ast.literal_eval)
df['votes'] = df['votes'].apply(ast.literal_eval)
df['predictions'] = df['predictions'].apply(ast.literal_eval)

# Convert lists in 'options' to tuples for hashability
df['options'] = df['options'].apply(tuple)

# Function to preprocess and convert rankings to indices
def convert_rankings_to_indices(rankings, options):
    option_to_index = {option: i + 1 for i, option in enumerate(sorted(options))}
    return [option_to_index[item] for item in rankings]

# Group by options
grouped = df.groupby('options')

# Dictionary to store models and fits for each option set
models = {}
fits = {}
rankings = {}

# Stan model code (modified for both votes and predictions)
stan_model_code = """
functions {
    real plackett_luce_lpmf(int[] ranking, vector strengths, int[] ground_truth) {
        real log_p = 0;
        for (k in 1:size(ranking)) {
            int item = ground_truth[ranking[k]];
            real sum_strengths = 0;
            for (m in k:size(ranking)) {
                int idx = ground_truth[ranking[m]];
                sum_strengths += strengths[idx];
            }
            log_p += log(strengths[item] / sum_strengths);
        }
        return log_p;
    }
}

data {
    int<lower=1> J;  // Number of questions
    int<lower=1> K;  // Number of options per question
    int<lower=1> N;  // Total number of unique options
    int<lower=1, upper=N> vote_indices[J, K];  // Indices of votes
    int<lower=1, upper=N> prediction_indices[J, K];  // Indices of predictions
    int<lower=1, upper=N> ground_truth_indices[J, K];  // Indices of ground truth
}

parameters {
    simplex[N] strengths_expert_vote;     // Normalized strength parameters for expert votes
    simplex[N] strengths_intermediate_vote; // Normalized strength parameters for intermediate votes
    simplex[N] strengths_nonexpert_vote;  // Normalized strength parameters for non-expert votes

    simplex[N] strengths_expert_prediction;     // Normalized strength parameters for expert predictions
    simplex[N] strengths_intermediate_prediction; // Normalized strength parameters for intermediate predictions
    simplex[N] strengths_nonexpert_prediction;  // Normalized strength parameters for non-expert predictions

    simplex[3] pi;                   // Mixing proportions for expert, intermediate, and non-expert
}

model {
    // Priors for votes (different for expert, intermediate, and non-expert)
    strengths_expert_vote ~ dirichlet(rep_vector(3, N));
    strengths_intermediate_vote ~ dirichlet(rep_vector(2, N));
    strengths_nonexpert_vote ~ dirichlet(rep_vector(1, N));

    // Priors for predictions (different for expert, intermediate, and non-expert)
    strengths_expert_prediction ~ dirichlet(rep_vector(1, N));
    strengths_intermediate_prediction ~ dirichlet(rep_vector(1, N));
    strengths_nonexpert_prediction ~ dirichlet(rep_vector(1, N));

    pi ~ dirichlet([1, 2, 3]');

    // Prior constraints for votes
    for (i in 1:(N-1)) {
        target += normal_lpdf(strengths_expert_vote[i+1] | strengths_expert_vote[i] - 0.05, 0.025);
        target += normal_lpdf(sum(strengths_expert_vote[1:i]) | sum(strengths_intermediate_vote[1:i]) + (N-i-1)*0.05, 0.025);
        target += normal_lpdf(sum(strengths_intermediate_vote[1:i]) | sum(strengths_nonexpert_vote[1:i]) + (N-i-1)*0.05, 0.025);

        // Prior constraints for predictions (same logic as votes)
        target += normal_lpdf(strengths_expert_prediction[i+1] | strengths_expert_prediction[i] - 0.05, 0.025);
        target += normal_lpdf(sum(strengths_expert_prediction[1:i]) | sum(strengths_intermediate_prediction[1:i]) + (N-i-1)*0.05, 0.025);
        target += normal_lpdf(sum(strengths_intermediate_prediction[1:i]) | sum(strengths_nonexpert_prediction[1:i]) + (N-i-1)*0.05, 0.025);
    }

    // Likelihood for both votes and predictions
    for (j in 1:J) {
        vector[3] log_probs_votes;
        vector[3] log_probs_predictions;

        // Likelihood for votes
        log_probs_votes[1] = log(pi[1]) + plackett_luce_lpmf(vote_indices[j] | strengths_expert_vote, ground_truth_indices[j]);
        log_probs_votes[2] = log(pi[2]) + plackett_luce_lpmf(vote_indices[j] | strengths_intermediate_vote, ground_truth_indices[j]);
        log_probs_votes[3] = log(pi[3]) + plackett_luce_lpmf(vote_indices[j] | strengths_nonexpert_vote, ground_truth_indices[j]);

        // Likelihood for predictions
        log_probs_predictions[1] = log(pi[1]) + plackett_luce_lpmf(prediction_indices[j] | strengths_expert_prediction, ground_truth_indices[j]);
        log_probs_predictions[2] = log(pi[2]) + plackett_luce_lpmf(prediction_indices[j] | strengths_intermediate_prediction, ground_truth_indices[j]);
        log_probs_predictions[3] = log(pi[3]) + plackett_luce_lpmf(prediction_indices[j] | strengths_nonexpert_prediction, ground_truth_indices[j]);

        // Adding the target likelihoods for both votes and predictions
        target += log_sum_exp(log_probs_votes);
        target += log_sum_exp(log_probs_predictions);
    }
}

generated quantities {
    vector[J] log_lik_votes;
    vector[J] log_lik_predictions;
    for (j in 1:J) {
        vector[3] log_probs_votes;
        vector[3] log_probs_predictions;

        // Generate log likelihood for votes
        log_probs_votes[1] = log(pi[1]) + plackett_luce_lpmf(vote_indices[j] | strengths_expert_vote, ground_truth_indices[j]);
        log_probs_votes[2] = log(pi[2]) + plackett_luce_lpmf(vote_indices[j] | strengths_intermediate_vote, ground_truth_indices[j]);
        log_probs_votes[3] = log(pi[3]) + plackett_luce_lpmf(vote_indices[j] | strengths_nonexpert_vote, ground_truth_indices[j]);

        // Generate log likelihood for predictions
        log_probs_predictions[1] = log(pi[1]) + plackett_luce_lpmf(prediction_indices[j] | strengths_expert_prediction, ground_truth_indices[j]);
        log_probs_predictions[2] = log(pi[2]) + plackett_luce_lpmf(prediction_indices[j] | strengths_intermediate_prediction, ground_truth_indices[j]);
        log_probs_predictions[3] = log(pi[3]) + plackett_luce_lpmf(prediction_indices[j] | strengths_nonexpert_prediction, ground_truth_indices[j]);

        log_lik_votes[j] = log_sum_exp(log_probs_votes);
        log_lik_predictions[j] = log_sum_exp(log_probs_predictions);
    }
}

"""

for options, group in grouped:
    # Apply the function to convert rankings to indices
    group['vote_indices'] = group['votes'].apply(lambda x: convert_rankings_to_indices(x, options))
    group['prediction_indices'] = group['predictions'].apply(lambda x: convert_rankings_to_indices(x, options))
    group['ground_truth_indices'] = group['options'].apply(lambda x: convert_rankings_to_indices(x, options))

    # Prepare data for the Stan model
    stan_data = {
        'J': len(group),
        'K': len(options),
        'N': len(options),
        'vote_indices': np.array(group['vote_indices'].tolist()),
        'prediction_indices': np.array(group['prediction_indices'].tolist()),
        'ground_truth_indices': np.array(group['ground_truth_indices'].tolist())
    }

    # Compile and fit the model for each group
    sm = pystan.StanModel(model_code=stan_model_code)
    fit = sm.sampling(data=stan_data, iter=6000, warmup=2000, chains=4)
    fits[str(options)] = fit

    # Extract the median of the posterior distributions for vote strengths
    expert_vote_strengths = np.median(fit['strengths_expert_vote'], axis=0)
    intermediate_vote_strengths = np.median(fit['strengths_intermediate_vote'], axis=0)
    nonexpert_vote_strengths = np.median(fit['strengths_nonexpert_vote'], axis=0)

    # Extract the median of the posterior distributions for prediction strengths
    expert_prediction_strengths = np.median(fit['strengths_expert_prediction'], axis=0)
    intermediate_prediction_strengths = np.median(fit['strengths_intermediate_prediction'], axis=0)
    nonexpert_prediction_strengths = np.median(fit['strengths_nonexpert_prediction'], axis=0)

    # Generate rankings based on vote strengths
    expert_vote_ranking = sorted(zip(options, expert_vote_strengths), key=lambda x: x[1], reverse=True)
    intermediate_vote_ranking = sorted(zip(options, intermediate_vote_strengths), key=lambda x: x[1], reverse=True)
    nonexpert_vote_ranking = sorted(zip(options, nonexpert_vote_strengths), key=lambda x: x[1], reverse=True)

    # Generate rankings based on prediction strengths
    expert_prediction_ranking = sorted(zip(options, expert_prediction_strengths), key=lambda x: x[1], reverse=True)
    intermediate_prediction_ranking = sorted(zip(options, intermediate_prediction_strengths), key=lambda x: x[1], reverse=True)
    nonexpert_prediction_ranking = sorted(zip(options, nonexpert_prediction_strengths), key=lambda x: x[1], reverse=True)

    # Store rankings for votes and predictions
    rankings[str(options)] = {
        'expert_vote_ranking': [x[0] for x in expert_vote_ranking],
        'intermediate_vote_ranking': [x[0] for x in intermediate_vote_ranking],
        'nonexpert_vote_ranking': [x[0] for x in nonexpert_vote_ranking],
        'expert_prediction_ranking': [x[0] for x in expert_prediction_ranking],
        'intermediate_prediction_ranking': [x[0] for x in intermediate_prediction_ranking],
        'nonexpert_prediction_ranking': [x[0] for x in nonexpert_prediction_ranking]
    }

# Print rankings for each group
for options, ranks in rankings.items():
    print(f"Options set: {options}")
    print(f"Expert Vote Rankings: {ranks['expert_vote_ranking']}")
    print(f"Intermediate Vote Rankings: {ranks['intermediate_vote_ranking']}")
    print(f"Non-Expert Vote Rankings: {ranks['nonexpert_vote_ranking']}")
    print(f"Expert Prediction Rankings: {ranks['expert_prediction_ranking']}")
    print(f"Intermediate Prediction Rankings: {ranks['intermediate_prediction_ranking']}")
    print(f"Non-Expert Prediction Rankings: {ranks['nonexpert_prediction_ranking']}")
    print()

# Optionally, convert the fit object to arviz InferenceData
for options, fit in fits.items():
    idata = az.from_pystan(posterior=fit)
    models[str(options)] = idata

In [ ]:
import pandas as pd
import numpy as np
import random
from scipy.stats import norm
from itertools import permutations
from typing import List, Tuple
from tqdm import tqdm
import ast  # to parse keys from fits

# -----------------------------
# Define Simulation Classes
# -----------------------------

class Vote:
    def __init__(self, question_number: int, options: List[int], ranking: List[int], predicted_probs: dict, group: int):
        self.question_number = question_number
        self.options = options
        self.ranking = ranking
        self.predicted_probs = predicted_probs
        self.group = group

class Voter:
    def __init__(self, group: int, strength_templates: List[List[float]], variances: List[float]):
        self.group = group  # 0 = expert, 1 = intermediate, 2 = non-expert
        self.strength_params = self.sample_strength_params(strength_templates[group], variances[group])

    def sample_strength_params(self, central_strengths: List[float], variance: float):
        strengths = [abs(norm.rvs(loc=s, scale=variance)) for s in central_strengths]
        total_strength = sum(strengths)
        return [s / total_strength for s in strengths]

    def vote(self, question_number: int, options: List[int], ground_truth: List[int], all_worlds: List[Tuple[int]]):
        signal = self.plackett_luce_vote(options)
        conditional_probs = {
            s_j: compute_conditional_prob(s_j, tuple(signal), all_worlds, ground_truth, self.group, self.strength_params)
            for s_j in all_worlds
        }
        predicted_probs = self.predict(signal, conditional_probs, all_worlds)
        prediction = max(predicted_probs, key=predicted_probs.get)
        return Vote(question_number, options, signal, prediction, self.group)

    def plackett_luce_vote(self, options):
        remaining_options = options.copy()
        ranking = []
        while remaining_options:
            current_strengths = [self.get_strength(opt) for opt in remaining_options]
            probs = [strength / sum(current_strengths) for strength in current_strengths]
            chosen = np.random.choice(remaining_options, p=probs)
            ranking.append(chosen)
            remaining_options.remove(chosen)
        return ranking

    def get_strength(self, option):
        return self.strength_params[option % len(self.strength_params)]

    def predict(self, signal, conditional_probs, all_worlds):
        prediction_probs = {world: prob for world, prob in conditional_probs.items() if world != tuple(signal)}
        total_prob = sum(prediction_probs.values())
        return {world: prob / total_prob for world, prob in prediction_probs.items()}


# -----------------------------
# Supporting Functions
# -----------------------------

computed_posteriors = {}

def compute_posterior(signal, world, all_worlds, ground_truth, group, strength_params):
    key = (tuple(signal), tuple(world), tuple(ground_truth), group)
    if key in computed_posteriors:
        return computed_posteriors[key]
    prior = 1 / len(all_worlds)
    likelihood = plackett_luce_probability(signal, world, strength_params)
    total_signal_prob = sum(plackett_luce_probability(signal, w, strength_params) * prior for w in all_worlds)
    posterior = likelihood * prior / total_signal_prob
    computed_posteriors[key] = posterior
    return posterior

def compute_conditional_prob(s_j, s_k, all_worlds, ground_truth, group, strength_params, num_samples=500):
    weights = [compute_posterior(s_k, w, all_worlds, ground_truth, group, strength_params) for w in all_worlds]
    sampled_worlds = random.choices(all_worlds, weights=weights, k=num_samples)
    return sum(plackett_luce_probability(s_j, w, strength_params) for w in sampled_worlds) / num_samples

def plackett_luce_probability(signal, world, strength_params):
    prob = 1.0
    remaining_options = list(world)
    for item in signal:
        if item in remaining_options:
            current_strengths = [strength_params[opt % len(strength_params)] for opt in remaining_options]
            total_strength = sum(current_strengths)
            idx = remaining_options.index(item)
            prob *= current_strengths[idx] / total_strength
            remaining_options.pop(idx)
    return prob

def simulate_voting(num_voters: int, subset: List[int], ground_truth: List[int],
                    strength_templates: List[List[float]], variances: List[float], mixing_probs: List[float]) -> List[Vote]:

    all_worlds = list(permutations(subset))
    question_number = 1
    votes = []

    group_assignments = np.random.choice([0, 1, 2], size=num_voters, p=mixing_probs)
    voters = [Voter(group=g, strength_templates=strength_templates, variances=variances) for g in group_assignments]

    for voter in tqdm(voters, desc="Simulating Votes"):
        vote = voter.vote(question_number, subset, ground_truth, all_worlds)
        votes.append(vote)

    return votes


# -----------------------------
# Main Generator from Fits
# -----------------------------

def generate_and_combine_synthetic_votes(fits_dict, num_voters, output_path):
    all_votes = []
    for key, fit in fits_dict.items():
        print(f"📦 Generating data for option set: {key}")
        try:
            # Median strength vectors for 3 groups
            expert_strengths = np.median(fit['strengths_expert_vote'], axis=0).tolist()
            intermediate_strengths = np.median(fit['strengths_intermediate_vote'], axis=0).tolist()
            nonexpert_strengths = np.median(fit['strengths_nonexpert_vote'], axis=0).tolist()

            # Normalize
            strength_templates = [
                [s / sum(expert_strengths) for s in expert_strengths],
                [s / sum(intermediate_strengths) for s in intermediate_strengths],
                [s / sum(nonexpert_strengths) for s in nonexpert_strengths],
            ]

            # Variances
            variances = [
                np.mean(np.var(fit['strengths_expert_vote'], axis=0)),
                np.mean(np.var(fit['strengths_intermediate_vote'], axis=0)),
                np.mean(np.var(fit['strengths_nonexpert_vote'], axis=0)),
            ]

            # Mixing proportions
            pi_median = np.median(fit['pi'], axis=0).tolist()
            mixing_probs = [p / sum(pi_median) for p in pi_median]

            subset = list(ast.literal_eval(key))
            ground_truth = sorted(subset)

            votes = simulate_voting(
                num_voters=num_voters,
                subset=subset,
                ground_truth=ground_truth,
                strength_templates=strength_templates,
                variances=variances,
                mixing_probs=mixing_probs
            )

            for vote in votes:
                all_votes.append([
                    vote.question_number,
                    vote.options,
                    vote.ranking,
                    vote.predicted_probs,
                    vote.group,
                    "Synthetic",
                    "PL"
                ])

        except Exception as e:
            print(f"⚠️ Skipping {key} due to error: {e}")

    df = pd.DataFrame(all_votes, columns=['question', 'options', 'votes', 'predictions', 'group', 'domain', 'treatment'])
    df.to_csv(output_path, index=False)
    print(f"\n✅ All synthetic votes saved to {output_path}")


# -----------------------------
# Execute Simulation
# -----------------------------

if __name__ == "__main__":
    output_path = "group3_plackett_luce_synthetic_votes.csv"
    num_voters = 1000

    generate_and_combine_synthetic_votes(
        fits_dict=fits,
        num_voters=num_voters,
        output_path=output_path
    )


In [ ]:
import pandas as pd
import numpy as np
import ast
import pystan
import arviz as az

# Load the data
df = pd.read_csv('group3_plackett_luce_synthetic_votes.csv')

# Convert string representations of lists into actual Python lists
df['options'] = df['options'].apply(ast.literal_eval)
df['votes'] = df['votes'].apply(ast.literal_eval)
df['predictions'] = df['predictions'].apply(ast.literal_eval)

# Convert lists in 'options' to tuples for hashability
df['options'] = df['options'].apply(tuple)

# Function to preprocess and convert rankings to indices
def convert_rankings_to_indices(rankings, options):
    option_to_index = {option: i + 1 for i, option in enumerate(sorted(options))}
    return [option_to_index[item] for item in rankings]

# Group by options
grouped = df.groupby('options')

# Dictionary to store models and fits for each option set
models = {}
synth_fits = {}
rankings = {}

# Stan model code adapted for 3 groups
stan_model_code = """
functions {
    real plackett_luce_lpmf(int[] ranking, vector strengths, int[] ground_truth) {
        real log_p = 0;
        for (k in 1:size(ranking)) {
            int item = ground_truth[ranking[k]];
            real sum_strengths = 0;
            for (m in k:size(ranking)) {
                int idx = ground_truth[ranking[m]];
                sum_strengths += strengths[idx];
            }
            log_p += log(strengths[item] / sum_strengths);
        }
        return log_p;
    }
}

data {
    int<lower=1> J;
    int<lower=1> K;
    int<lower=1> N;
    int<lower=1, upper=N> vote_indices[J, K];
    int<lower=1, upper=N> prediction_indices[J, K];
    int<lower=1, upper=N> ground_truth_indices[J, K];
}

parameters {
    simplex[N] strengths_expert_vote;
    simplex[N] strengths_intermediate_vote;
    simplex[N] strengths_nonexpert_vote;
    simplex[N] strengths_expert_prediction;
    simplex[N] strengths_intermediate_prediction;
    simplex[N] strengths_nonexpert_prediction;
    simplex[3] pi;
}

model {
    strengths_expert_vote ~ dirichlet(rep_vector(3, N));
    strengths_intermediate_vote ~ dirichlet(rep_vector(2, N));
    strengths_nonexpert_vote ~ dirichlet(rep_vector(1, N));
    strengths_expert_prediction ~ dirichlet(rep_vector(1, N));
    strengths_intermediate_prediction ~ dirichlet(rep_vector(1, N));
    strengths_nonexpert_prediction ~ dirichlet(rep_vector(1, N));

    pi ~ dirichlet([1, 2, 3]');

    for (i in 1:(N-1)) {
        target += normal_lpdf(strengths_expert_vote[i+1] | strengths_expert_vote[i] - 0.05, 0.025);
        target += normal_lpdf(sum(strengths_expert_vote[1:i]) | sum(strengths_intermediate_vote[1:i]) + (N-i-1)*0.05, 0.025);
        target += normal_lpdf(sum(strengths_intermediate_vote[1:i]) | sum(strengths_nonexpert_vote[1:i]) + (N-i-1)*0.05, 0.025);

        target += normal_lpdf(strengths_expert_prediction[i+1] | strengths_expert_prediction[i] - 0.05, 0.025);
        target += normal_lpdf(sum(strengths_expert_prediction[1:i]) | sum(strengths_intermediate_prediction[1:i]) + (N-i-1)*0.05, 0.025);
        target += normal_lpdf(sum(strengths_intermediate_prediction[1:i]) | sum(strengths_nonexpert_prediction[1:i]) + (N-i-1)*0.05, 0.025);
    }

    for (j in 1:J) {
        vector[3] log_probs_votes;
        vector[3] log_probs_predictions;

        log_probs_votes[1] = log(pi[1]) + plackett_luce_lpmf(vote_indices[j] | strengths_expert_vote, ground_truth_indices[j]);
        log_probs_votes[2] = log(pi[2]) + plackett_luce_lpmf(vote_indices[j] | strengths_intermediate_vote, ground_truth_indices[j]);
        log_probs_votes[3] = log(pi[3]) + plackett_luce_lpmf(vote_indices[j] | strengths_nonexpert_vote, ground_truth_indices[j]);

        log_probs_predictions[1] = log(pi[1]) + plackett_luce_lpmf(prediction_indices[j] | strengths_expert_prediction, ground_truth_indices[j]);
        log_probs_predictions[2] = log(pi[2]) + plackett_luce_lpmf(prediction_indices[j] | strengths_intermediate_prediction, ground_truth_indices[j]);
        log_probs_predictions[3] = log(pi[3]) + plackett_luce_lpmf(prediction_indices[j] | strengths_nonexpert_prediction, ground_truth_indices[j]);

        target += log_sum_exp(log_probs_votes);
        target += log_sum_exp(log_probs_predictions);
    }
}

generated quantities {
    vector[J] log_lik_votes;
    vector[J] log_lik_predictions;
    for (j in 1:J) {
        vector[3] log_probs_votes;
        vector[3] log_probs_predictions;

        log_probs_votes[1] = log(pi[1]) + plackett_luce_lpmf(vote_indices[j] | strengths_expert_vote, ground_truth_indices[j]);
        log_probs_votes[2] = log(pi[2]) + plackett_luce_lpmf(vote_indices[j] | strengths_intermediate_vote, ground_truth_indices[j]);
        log_probs_votes[3] = log(pi[3]) + plackett_luce_lpmf(vote_indices[j] | strengths_nonexpert_vote, ground_truth_indices[j]);

        log_probs_predictions[1] = log(pi[1]) + plackett_luce_lpmf(prediction_indices[j] | strengths_expert_prediction, ground_truth_indices[j]);
        log_probs_predictions[2] = log(pi[2]) + plackett_luce_lpmf(prediction_indices[j] | strengths_intermediate_prediction, ground_truth_indices[j]);
        log_probs_predictions[3] = log(pi[3]) + plackett_luce_lpmf(prediction_indices[j] | strengths_nonexpert_prediction, ground_truth_indices[j]);

        log_lik_votes[j] = log_sum_exp(log_probs_votes);
        log_lik_predictions[j] = log_sum_exp(log_probs_predictions);
    }
}
"""

# Loop through each group
for options, group in grouped:
    group['vote_indices'] = group['votes'].apply(lambda x: convert_rankings_to_indices(x, options))
    group['prediction_indices'] = group['predictions'].apply(lambda x: convert_rankings_to_indices(x, options))
    group['ground_truth_indices'] = group['options'].apply(lambda x: convert_rankings_to_indices(x, options))

    stan_data = {
        'J': len(group),
        'K': len(options),
        'N': len(options),
        'vote_indices': np.array(group['vote_indices'].tolist()),
        'prediction_indices': np.array(group['prediction_indices'].tolist()),
        'ground_truth_indices': np.array(group['ground_truth_indices'].tolist())
    }

    sm = pystan.StanModel(model_code=stan_model_code)
    synth_fit = sm.sampling(data=stan_data, iter=2000, chains=4)
    synth_fits[str(options)] = synth_fit

    expert_strengths = np.median(synth_fit['strengths_expert_vote'], axis=0)
    intermediate_strengths = np.median(synth_fit['strengths_intermediate_vote'], axis=0)
    nonexpert_strengths = np.median(synth_fit['strengths_nonexpert_vote'], axis=0)

    expert_ranking = sorted(zip(options, expert_strengths), key=lambda x: x[1], reverse=True)
    intermediate_ranking = sorted(zip(options, intermediate_strengths), key=lambda x: x[1], reverse=True)
    nonexpert_ranking = sorted(zip(options, nonexpert_strengths), key=lambda x: x[1], reverse=True)

    rankings[str(options)] = {
        'expert_ranking': [x[0] for x in expert_ranking],
        'intermediate_ranking': [x[0] for x in intermediate_ranking],
        'nonexpert_ranking': [x[0] for x in nonexpert_ranking]
    }

    print(f"Options set: {options}")
    print(f"Expert Rankings: {rankings[str(options)]['expert_ranking']}")
    print(f"Intermediate Rankings: {rankings[str(options)]['intermediate_ranking']}")
    print(f"Non-Expert Rankings: {rankings[str(options)]['nonexpert_ranking']}")
    print()

    idata = az.from_pystan(posterior=synth_fit)
    models[str(options)] = idata


In [ ]:
def extract_mean_params(fit):
    summary = fit.summary()
    param_names = summary['summary_rownames']
    means = summary['summary'][:, 0]  # First column is the mean
    return dict(zip(param_names, means))


In [ ]:
import pandas as pd

comparison_records = []

for key in fits.keys():
    print(f"\n📊 Comparing parameter recovery for option set: {key}")
    try:
        original_params = extract_mean_params(fits[key])
        synthetic_params = extract_mean_params(synth_fits[key])

        # Determine the number of options (N) from any strength param group
        N = sum(1 for p in original_params if p.startswith('strengths_expert_vote'))

        # Compare mixing proportions
        for i in range(1, 4):  # pi[1], pi[2], pi[3]
            param = f'pi[{i}]'
            rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
            comparison_records.append({
                'options': key,
                'parameter': param,
                'original': original_params[param],
                'synthetic': synthetic_params[param],
                'relative_error': rel_error
            })
            print(f"{param}: Relative Error = {rel_error:.4f}")

        # Compare all strength parameters
        for group in ['expert_vote', 'intermediate_vote', 'nonexpert_vote',
                      'expert_prediction', 'intermediate_prediction', 'nonexpert_prediction']:
            for i in range(N):
                param = f'strengths_{group}[{i+1}]'
                rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
                comparison_records.append({
                    'options': key,
                    'parameter': param,
                    'original': original_params[param],
                    'synthetic': synthetic_params[param],
                    'relative_error': rel_error
                })
                print(f"{param}: Relative Error = {rel_error:.4f}")

    except Exception as e:
        print(f"⚠️ Could not compare for {key}: {e}")


In [ ]:
df_compare = pd.DataFrame(comparison_records)
df_compare.to_csv('parameter_recovery_comparison_3groups.csv', index=False)


In [ ]:
import numpy as np
import pandas as pd

# Initialize dictionaries to store relative errors for each parameter group
relative_errors = {
    'expert_vote': [],
    'intermediate_vote': [],
    'nonexpert_vote': [],
    'expert_prediction': [],
    'intermediate_prediction': [],
    'nonexpert_prediction': [],
    'expert_proportion': [],
    'intermediate_proportion': [],
    'nonexpert_proportion': []
}

# Iterate over each group of options
for key in fits.keys():
    try:
        original_params = extract_mean_params(fits[key])
        synthetic_params = extract_mean_params(synth_fits[key])

        # Determine the number of options (N)
        N = sum(1 for p in original_params if p.startswith('strengths_expert_vote'))

        # Compute relative errors for each parameter group
        for i in range(N):
            for group in ['expert', 'intermediate', 'nonexpert']:
                for mode in ['vote', 'prediction']:
                    param = f'strengths_{group}_{mode}[{i+1}]'
                    rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
                    relative_errors[f'{group}_{mode}'].append(rel_error)

        # Group proportions
        for i, group in enumerate(['expert', 'intermediate', 'nonexpert']):
            param = f'pi[{i+1}]'
            rel_error = abs(original_params[param] - synthetic_params[param]) / original_params[param]
            relative_errors[f'{group}_proportion'].append(rel_error)

    except Exception as e:
        print(f"⚠️ Could not compare for {key}: {e}")


In [ ]:
# Compute average relative errors
average_errors = {param: np.mean(errors) for param, errors in relative_errors.items()}

# Display the results
print("\n📊 Aggregate Parameter Recovery Errors:")
for param_group, avg_error in average_errors.items():
    print(f"🔹 {param_group.replace('_', ' ').title()}: {avg_error:.4f}")


In [ ]:
# Convert the results to a DataFrame
df_summary = pd.DataFrame([
    {'Parameter Group': param_group.replace('_', ' ').title(), 'Average Relative Error': avg_error}
    for param_group, avg_error in average_errors.items()
])

# Save to CSV
df_summary.to_csv('aggregate_parameter_recovery_summary_3.csv', index=False)
